In [ ]:
import plotly.express as px
import pandas as pd
import requests
import arrow
import sys # added!
sys.path.append("../app") # added!

In [ ]:
from io import StringIO

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/gov.db')


In [ ]:
# TODO: convert to dictionary to name

from config import TRUST_INFO

YESTERDAY = str(arrow.now().shift(days=-1).format("YYYY-MM-DD"))

In [ ]:
URL_HOSP_CASES = f"https://coronavirus.data.gov.uk/api/v2/data?areaType=nhsTrust&release={YESTERDAY}&metric=hospitalCases&format=json"

In [ ]:
URL_CASES_BY_AGE = f"https://api.coronavirus.data.gov.uk/v2/data?areaType=region&areaCode=E12000007&metric=newCasesBySpecimenDateAgeDemographics&format=csv"

In [ ]:
def request_gov_uk(url, table, engine, format='json') -> pd.DataFrame:
    """
    Import COVID information as per the gov.uk API here
    but checks to see if the same request has already been run
    url: API connection
    table: table to store data in local SQLite
    conn: connection to local db for storing data and logging requests
    format: json or csv
    """
    with engine.connect() as conn:
        requests_df = pd.read_sql('requests_log', conn)

        if url in requests_df.request.values:
            df = pd.read_sql(table, conn)
        
        else:
            response = requests.get(url)
            if format == 'json':
                df = pd.json_normalize(response.json(), record_path="body")
            elif format == 'csv':
                df = pd.read_csv(StringIO(response.text))
            else:
                raise NotImplementedError
            df.to_sql(table, conn, if_exists='replace')

            request_log = pd.DataFrame({
                'request': [url],
                'table': [table],
                'request_ts': [str(arrow.now())]
            })
            request_log.to_sql('requests_log', conn, if_exists='append')
    return df


In [ ]:
df = request_gov_uk(URL_CASES_BY_AGE, 'cases_by_age', engine, format='csv')

In [ ]:
df

In [ ]:
def clean_popn_cases(df):
    """
    Clean population cases (London)
    df : data frame of population cases by age
    """
    df["date"] = pd.to_datetime(df["date"])
    df.drop(["areaType", "areaCode", "areaType" ], axis=1, inplace=True)
    return df

In [ ]:
df = clean_popn_cases(df)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

In [ ]:
age_bands = df.age.unique().tolist()
age_bands = list(set(age_bands) - set(['60+', '00_59', 'unassigned']))
age_bands

In [ ]:
for age in age_bands:
    _df = df.loc[df.age == age]
    fig.add_trace(
            go.Scatter(
                name=age,
                x=_df.date,
                y=_df.cases,
            )
        )

In [ ]:
fig

In [ ]:
fig.add_trace(go.Scatter(x=df.date, y=df.cases, fill=df.age))

In [ ]:
df.head()

In [ ]:
def clean_hosp_cases(df, TRUST_INFO):
    """
    Clean hosp cases
    df : data frame of hospital cases
    TRUST_INFO: data frame of trust information
    """
    trusts_london = TRUST_INFO[TRUST_INFO.inLondon01][['areaCode','shortName', 'sectorName', 'inNCL01']]
    df = df.merge(trusts_london, how='inner', on='areaCode')
    df["date"] = pd.to_datetime(df["date"])
    df.drop(['index', 'areaType'], axis=1, inplace=True)
    return df


In [ ]:
df2 = clean_hosp_cases(df, TRUST_INFO)

In [ ]:
df2.areaCode.unique().tolist()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

In [ ]:
fig.add_trace(go.Scatter(x=df.date, y=df.hospitalCases))

In [ ]:
df

In [ ]:
df.loc[df.hospitalCases==49]